# Sentiment-Analyse für Filmrezensionen

In diesem Notebook beschäftigen wir uns mit Filmrezensionen, die in der IMDB öffentlich verfügbar sind. Es handelt sich hierbei um natürlichsprachliche (englische) Texte und unsere Aufgabe ist es, an Hand des Texts zu erkennen, ob der Tenor der Rezension positiv oder negativ ist.

## Laden der Bibliotheken und Daten und Definition von Hilfsvariablen

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import imdb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Die folgenden Variablen dienen uns dazu, zwischen Wörtern und deren Kodierung als Zahlen zu wechseln.

In [ ]:
word_index = imdb.get_word_index()
index_lookup = dict((v, k) for k, v  in word_index.items())

## Blick in die Daten

Wir schauen uns nun beispielhaft ein paar Rezensionen an. Ändern Sie den Wert der Variablen i, um weitere Rezensionen auszugeben. Über der Rezension wird angezeigt, ob es sich um eine positive ("Good review") oder negative ("Bad review") Rezension handelt.

In [ ]:
i = 0
print('Good' if train_labels[i] else 'Bad', 'review\n')
print(' '.join([index_lookup.get(w - 3, '?') for w in train_data[i]]))

## One-Hot-Encoding der Daten

Nun wird ein sogenanntes One-Hot-Encoding durchgeführt. Im Ergebnis erhalten wir für jede Rezension einen Vektor von Nullen und Einsen, der wiedergibt, welche Wörter in der Rezension vorkommen (jedoch nicht ihre Reihenfolge). Dieser Ansatz wird als "Bag of Words" bezeichnet.

In [ ]:
def one_hot_encoding(sequences, dimension=10000):
    res = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            res[i, j] = 1
    
    return res

X_train = one_hot_encoding(train_data)
X_test = one_hot_encoding(test_data)

Beispielhaft schauen wir uns das Verfahren für die Rezension Nr. 5 an

In [ ]:
print('\n'.join(f'{w}: {index_lookup.get(w - 3, "?")}' for w in range(20)))
print()
print(train_data[5])
print(' '.join([index_lookup.get(w - 3, '?') for w in train_data[5]]))

Nun geben wir die ersten Hundert Einträge des entsprechend One-Hot-Vektors aus. Wir sehen, dass genau dort eine Eins steht, wo das entsprechend Wort in der Rezension vorkommt. Beispielhaft sehen wir an 5. Stelle eine Eins für das Wort "the" und an 9. Stelle eine Eins für "to". (Die ersten vier Einträge mit Fragezeichen entsprechen speziellen Steuerungsinformationen und können hier ignoriert werden)

In [ ]:
X_train[5, :100]

## Erstellung des Modells

Wir trainieren nun eine logistische Regression auf den Daten. Der Parameter C steuert dabei das Ausmaß der Regularisierung. Je kleiner C ist, desto stärker wird das Modell regularisiert. Erproben Sie gerne verschiedene Werte von C und vergleichen die Ergebnisse.

In [ ]:
# C is the regularization coefficient, where here smaller value imply stronger regularization.
# Feel free to play around with different values
lm = LogisticRegression(solver='liblinear', penalty='l1', C=.1)

lm.fit(X_train, train_labels)

### Ergebnisse auf dem Test Set

Wir überprüfen nun die Güte des Modells auf dem Testdatensatz

In [ ]:
test_pred = lm.predict(X_test)

In [ ]:
print(classification_report(test_labels, test_pred))

In [ ]:
print(confusion_matrix(test_labels, test_pred))

In [ ]:
_ = RocCurveDisplay.from_estimator(lm, X_test, test_labels)

### Analyse von aussagekräftigen Wörtern

Indem wir die Wörter nach ihren numerischen Koeffizienten in der Regression sortieren, können wir uns anschauen, welche Wörter im Modell eine besonders negative oder positive Konnotation haben.

In [ ]:
word_coefs = [(coef, index_lookup.get(w - 3, '?')) for w, coef in enumerate(lm.coef_[0])]

Besonders negative Wörter

In [ ]:
sorted(word_coefs)[:10]

Besonders positive Wörter

In [ ]:
sorted(word_coefs, reverse=True)[:10]

## Bonus: Neuronales Netz

Bei einer logistischen Regression handelt es sich um eine klassische, aber durchaus leistungsfähige Verfahrensklasse. Inzwischen hat sich für viele Anwendungen, u.a. die Sentiment Analyse, der Einsatz von Deep Learning etabliert. Wir erproben hier an Hand eines einfachen neuronalen Netzes diese Verfahrensklasse.

Zuerst laden wir die nötigen Bibliotheken und definieren die Struktur des neuronalen Netzes

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Nachdem wir nun das Modell trainiert, wobei beim Training der Datensatz 5 mal (entsprechend 5 Epochen) durchlaufen wird

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=5,
                    batch_size=512)

### Ergebnisse auf dem Test Set

Eine Auswertung auf dem Testdatensatz zeigt, dass zumindest mit diesem einfachen neuronalen Netzwerk die Ergebnisse ungefähr gleichauf liegen mit der logistischen Regression

In [ ]:
test_pred_nn = model.predict(X_test)[:, 0]

In [ ]:
print(classification_report(test_labels, test_pred_nn > 0.5))

In [ ]:
print(confusion_matrix(test_labels, test_pred_nn > 0.5))

In [ ]:
_ = RocCurveDisplay.from_predictions(test_labels, test_pred_nn)